# Example 5-4: Calculating Moonrise-Moonset Times
### _Fundamentals of Astrodynamics and Applications_, 5th Ed., 2022, p. 298-300

This notebook demonstrates calculating the moonrise and moonset times given a date and site location.

## Install and Import Libraries
---

First, install `valladopy` if it doesn't already exist in your environment:

In [1]:
!pip install valladopy==0.4.1

Import `numpy` and the relevant `valladopy` modules:

In [2]:
import numpy as np
import valladopy.constants as const
from valladopy.astro.celestial import moon
from valladopy.mathtime.julian_date import jday
from valladopy.mathtime.utils import sec2hms

## Problem Definition
---

GIVEN:&ensp;Site at $40^{\circ}\text{N}$, $0^{\circ}\text{E}$ on August 21, 1998<br>
FIND: &emsp;Moonrise and moonset times

In [3]:
year = 1998
month = 8
day = 21
latgd = np.radians(40)
lon = 0

## Solution
---

We begin by finding the Julian Date using **Algorithm 14**:

$$
JD = 367(yr)
- \text{INT}\left\{ \frac{7\left\{ yr + \text{INT} \left( \frac{mo + 9}{12} \right) \right\}}{4} \right\}
+ \text{INT}\left( \frac{275\,mo}{9} \right)
+ d + 1{,}721{,}013.5
+ \frac{h\,3600 + min\,60 + s}{86400}
$$

Which is done with the `jday` routine:

In [4]:
jd, jdfrac = jday(year, month, day)

print(f'Julian Date:\t{jd + jdfrac}')

Julian Date:	2451046.5


**Algorithm 32** shows the process for calculating moonrise and moonset times separately.

Begin by finding the Julian dates of the moonrise and moonset occurrences, using the JD at $0^{\text{h}}$:

$$
\begin{aligned}
JD_{moonrise} &= JD_{0^{\text{h}}} + 6/24 - \lambda/360 \\
\\
JD_{moonset} &= JD_{0^{\text{h}}} + 18/24 - \lambda/360
\end{aligned}
$$

Find the Julian centuries of TDB (letting $TDB \cong TT$):

$$
T_{TDB} = \frac{JD_{TDB} - 2,\!451,\!545.0}{36,\!525}
$$

Then, find the ecliptic longitude and latitude of the moon:

$$
\begin{align*}
\lambda_{\text{ecliptic ☾}} &= 218.32^\circ + 481,\!267.8813\,T_{TDB} \\
&\quad + 6.29 \sin(134.9^\circ + 477,\!198.85\,T_{TDB}) \\
&\quad - 1.27 \sin(259.2^\circ - 413,\!335.38\,T_{TDB}) \\
&\quad + 0.66 \sin(235.7^\circ + 890,\!534.23\,T_{TDB}) \\
&\quad + 0.21 \sin(269.9^\circ + 954,\!397.70\,T_{TDB}) \\
&\quad - 0.19 \sin(357.5^\circ + 35,\!999.05\,T_{TDB}) \\
&\quad - 0.11 \sin(186.6^\circ + 966,\!404.05\,T_{TDB})
\end{align*}
$$

$$
\begin{align*}
\phi_{\text{ecliptic ☾}} &= 5.13^\circ \sin(93.3^\circ + 483,\!202.03\,T_{TDB}) \\
&\quad + 0.28 \sin(228.2^\circ + 960,\!400.87\,T_{TDB}) \\
&\quad - 0.28 \sin(318.3^\circ + 6003.18\,T_{TDB}) \\
&\quad - 0.17 \sin(217.6^\circ - 407,\!332.20\,T_{TDB})
\end{align*}
$$

The obliquity of the ecliptic can be approximated as:

$$
\epsilon = 23.439291^{\circ} - 0.013004 \ T_{TDB} \ \cong \ 23.439291^{\circ} - 0.013004 \ T_{UT1}
$$

The direction cosines are:

$$
\begin{align*}
l &= \cos(\phi_{\text{ecliptic ☾}})\cos(\lambda_{\text{ecliptic ☾}}) \\
m &= \cos(\epsilon)\cos(\phi_{\text{ecliptic ☾}})\sin(\lambda_{\text{ecliptic ☾}})
     - \sin(\epsilon)\sin(\phi_{\text{ecliptic ☾}}) \\
n &= \sin(\epsilon)\cos(\phi_{\text{ecliptic ☾}})\sin(\lambda_{\text{ecliptic ☾}})
     + \cos(\epsilon)\sin(\phi_{\text{ecliptic ☾}})
\end{align*}
$$


And obtain the right ascension and declination with:

$$
\begin{align*}
\alpha_c &= \operatorname{atan2}(m,\, l) \\
\sin(\delta_c) &= n
\end{align*}
$$

Page 297 of **Algorithm 32** outlines the remaining steps in the loop to ultimately determine the universal times for moonrise and moonset.

All of this is encompassed in the `moon.rise_set` routine, which returns the moonrise and moonset universal times in **hours**:

In [5]:
moonrise, moonset, _ = moon.rise_set(jd + jdfrac, latgd, lon)

# Convert to HMS format:
moonrise_hms = sec2hms(moonrise * const.HR2SEC)
moonset_hms = sec2hms(moonset * const.HR2SEC)

print(f'Sunrise:\t{moonrise_hms[0]} hr,\t{moonrise_hms[1]} min,\t{moonrise_hms[2]:.4f} sec')
print(f'Sunset:\t\t{moonset_hms[0]} hr,\t{moonset_hms[1]} min,\t{moonset_hms[2]:.4f} sec')

Sunrise:	4 hr,	31 min,	7.6768 sec
Sunset:		18 hr,	31 min,	5.1069 sec


These values are close to those found in the Almanac (1998).